# COVID-19 Analysis
Guillaume Rozier, 2020

In [3]:
import requests
import json
from datetime import date
from datetime import datetime
import numpy as np
import sys
import chart_studio
import pandas as pd
import plotly.graph_objects as go
import plotly
import chart_studio.plotly as py

chart_studio.tools.set_credentials_file(username='worldice', api_key='2iXFe4Ch2oPo1dpaBj8p')

##### Functions

In [87]:
def compute_offset(df, col_of_reference, col_to_align):
    diffs = []
    for offset in range(len(df)):
        delta = df[col_of_reference][:].shift(offset) - df[col_to_align][:]
        diffs.append(abs(delta.mean()))
    ret = diffs.index(min(diffs))
    #print(ret)
    if col_of_reference == col_to_align:
        return 0
    return ret


### DATA

#### Download data

In [88]:
today = datetime.now().strftime("%Y-%m-%d %H:%M")

url_confirmed = "https://cowid.netlify.com/data/total_cases.csv"
url_deaths = "https://cowid.netlify.com/data/total_deaths.csv"
    
r_confirmed = requests.get(url_confirmed)
r_deaths = requests.get(url_deaths)

with open('data/total_cases_who.csv', 'wb') as f:
    f.write(r_confirmed.content)
    
with open('data/total_deaths_who.csv', 'wb') as f:
    f.write(r_deaths.content)

#### Import data and merge

In [100]:
df_confirmed_who = pd.read_csv('data/total_cases_who.csv')
df_deaths_who = pd.read_csv('data/total_deaths_who.csv')

df_confirmed_perso = pd.read_csv('data/total_cases_perso.csv')
df_deaths_perso = pd.read_csv('data/total_deaths_perso.csv')

#df_confirmed = pd.concat([df_confirmed_who, df_confirmed_perso], keys=['date'])
df_confirmed = pd.merge(df_confirmed_perso,df_confirmed_who, how='outer')
df_deaths = pd.merge(df_deaths_who, df_deaths_perso, how='outer')

In [84]:
df_confirmed

,date,France,Luxembourg,World,Afghanistan,Albania,Algeria,Andorra,Antigua and Barbuda,Argentina,...,Trinidad and Tobago,Tunisia,Turkey,Ukraine,United Arab Emirates,United Kingdom,United States,Vatican,Venezuela,Vietnam
0,2020-03-14,4500.0,52.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-01-21,NaN,NaN,282.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-01-22,NaN,NaN,314.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-01-23,NaN,NaN,581.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
4,2020-01-24,NaN,NaN,846.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,2.0
5,2020-01-25,3.0,NaN,1320.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,2.0
6,2020-01-26,3.0,NaN,2014.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,2.0
7,2020-01-27,3.0,NaN,2798.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,2.0
8,2020-01-28,3.0,NaN,4593.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,2.0
9,2020-01-29,4.0,NaN,6065.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,4.0,NaN,5.0,NaN,NaN,2.0


#### Informations on countries (population, offset)

In [95]:
# Importing informations on countries
with open('data/info_countries.json', 'r') as f:
    countries = json.load(f)
    
# Computing offset
for c in countries:
    countries[c]['offset'] = compute_offset(df_confirmed, 'Italy', c)

# Exporting informations on countries
with open('data/info_countries.json', 'w') as fp:
    json.dump(countries, fp)

# Graphs

### Total cases for 1 million inhabitants

In [101]:
fig = go.Figure()

last_d = len(df_confirmed)
      
for country in countries:
    fig.add_trace(go.Scatter(x=df_confirmed['date'][-last_d:], y=df_confirmed[country][-last_d:]/countries[country]['pop'],
                    mode='lines+markers',
                    name='{}'.format(country)))

fig.update_layout(
    title="COVID-19 total cases over time for 1 million inhabitants",
    xaxis_title="Time (day)",
    yaxis_title="COVID-19 total confirmed cases / nb of inhabitants (million)",
    annotations = [dict(xref='paper',
        yref='paper',
        x=0, y=1.1,
        showarrow=False,
        text ='Last update: {} ; Last data: {}'.format(today, df_confirmed['date'].values[-1]))]
)
fig.update_xaxes(nticks = last_d)
#plotly.offline.plot(fig, filename = 'cases.html', auto_open=False)
#fig.write_image('cases.png')
#py.plot(fig, filename = 'cases', auto_open=False)
fig.show()

### Total cases (world)

In [102]:
fig = go.Figure()

last_d = len(df_confirmed)

for col in df_confirmed.columns[2:]:
    fig.add_trace(go.Scatter(x=df_confirmed['date'][-last_d:], y=df_confirmed[col][-last_d:],
                    mode='lines+markers',
                    name='{}'.format(col)))

fig.update_layout(
    title="COVID-19 total cases over time",
    xaxis_title="Time (day)",
    yaxis_title="COVID-19 total confirmed cases",
    annotations = [dict(xref='paper',
        yref='paper',
        x=0, y=1.1,
        showarrow=False,
        text ='Last update: {} ; Last data: {}'.format(today, df_confirmed['date'].values[-1]))]
)
fig.update_xaxes(nticks = last_d)
plotly.offline.plot(fig, filename = 'cases.html', auto_open=False)
#fig.write_image('cases.png')
#py.plot(fig, filename = 'cases', auto_open=False)
fig.show()

### Total cases for 1 million inhabitants [aligned]

In [105]:
import plotly.graph_objects as go

fig = go.Figure()

last_d = 20
countries["Luxembourg"]["offset"] = 11
countries["Belgium"]["offset"] = 8

for c in countries:
    offset = countries[c]['offset']
    if offset==0: offset2=-1
    pop = countries[c]['pop']
    print(offset)
    fig.add_trace(go.Scatter(x=df_confirmed['date'][ -last_d - offset:], y=df_confirmed[c][-last_d:]/pop,
                    mode='lines+markers',
                    name='{} [delayed by {} days]'.format(c, -offset)))

fig.update_layout(
    title="COVID-19 total cases over time for 1 million inhabitants [aligned for comparison]",
    xaxis_title="Time (day) — delayed for some countries",
    yaxis_title="COVID-19 total confirmed cases / nb of inhabitants (million)",
    annotations = [dict(xref='paper',
        yref='paper',
        x=0, y=1.1,
        showarrow=False,
        text ='Last update: {} ; Last data: {}'.format(today, df_confirmed['date'].values[-1]))]
)


fig.update_xaxes(nticks = last_d)
plotly.offline.plot(fig, filename = 'cases_aligned.html', auto_open=False)
fig.show()
py.plot(fig, filename = 'cases-aligned', auto_open=False)
#py.iplot(fig, filename='covid_aligned.html')

8
0
13
9
9
8
11


'https://plot.ly/~worldice/7/'

### Total deaths for 1 million inhabitants

In [68]:
import plotly.graph_objects as go
fig = go.Figure()

last_d = len(df_deaths)

for c in countries:
    pop = countries[c]["pop"]
    fig.add_trace(go.Scatter(x=df_deaths['date'][-last_d:], y=df_deaths[c][-last_d:] / pop,
                    mode='lines+markers',
                    name='{}'.format(c)))

fig.update_layout(
    title="COVID-19 deaths over time for 1 million inhabitants",
    xaxis_title="Time (day)",
    yaxis_title="COVID-19 total deaths / nb of inhabitants (million)",
    annotations = [dict(xref='paper',
        yref='paper',
        x=0, y=1.1,
        showarrow=False,
        text ='Last update: {} ; Last data: {}'.format(today, df_confirmed['date'].values[-1]))]
)
fig.update_xaxes(nticks = last_d)
plotly.offline.plot(fig, filename = 'deaths.html', auto_open=False)
fig.show()
py.plot(fig, filename = 'deaths', auto_open=False)

KeyboardInterrupt: 

### Total deaths for 1 million inhabitants [aligned]

In [69]:
import plotly.graph_objects as go
import plotly

fig = go.Figure()

last_d = 16
offset_france = compute_offset(df_confirmed, 'Italy', 'France')
offset_uk = compute_offset(df_confirmed, 'Italy', 'United Kingdom')
offset_sp = compute_offset(df_confirmed, 'Italy', 'Spain')
offset_ge = compute_offset(df_confirmed, 'Italy', 'Germany')
upset_ch = 1

for c in countries:
    pop = countries[c]['pop']
    offset = countries[c]['offset']
    if offset==0: offset2=-1
    fig.add_trace(go.Scatter(x = df_deaths['date'][-last_d-offset:], y=df_deaths[c][-last_d:]/pop,
                    mode='lines+markers',
                    name='{} [delayed by {} days]'.format(c, -offset)))
    
"""
fig.add_trace(go.Scatter(x=df_deaths['date'][-last_d-offset_france:-offset_france], y=df_deaths['France'][-last_d:]/pop_fr,
                    mode='lines+markers',
                    name='France [delayed by {} days]'.format(-offset_france)))
fig.add_trace(go.Scatter(x=df_deaths['date'][-last_d:], y=df_deaths['Italy'][-last_d:]/pop_it,
                    mode='lines+markers',
                    name='Italy [reference]'))
fig.add_trace(go.Scatter(x=df_deaths['date'][-last_d-offset_uk:-offset_uk], y=df_deaths['United Kingdom'][-last_d:]/pop_uk,
                    mode='lines+markers',
                    name='UK [delayed by {} days]'.format(-offset_uk)))

fig.add_trace(go.Scatter(x=df_deaths['date'][-last_d:], y=df_deaths['China'][upset_ch : upset_ch+last_d]/pop_ch,
                    mode='lines+markers',
                    name='CH [delayed by {} days]'.format(len(df_confirmed)-upset_ch)))
fig.add_trace(go.Scatter(x=df_deaths['date'][-last_d-offset_sp:-offset_sp], y=df_deaths['Spain'][-last_d:]/pop_sp,
                    mode='lines+markers',
                    name='Spain [delayed by {} days]'.format(-offset_sp)))
fig.add_trace(go.Scatter(x=df_deaths['date'][-last_d-offset_ge:-offset_ge], y=df_deaths['Germany'][-last_d:]/pop_ge,
                    mode='lines+markers',
                    name='Germany [delayed by {} days]'.format(-offset_ge)))

"""
fig.update_layout(
    title="COVID-19 deaths over time for 1 million inhabitants [aligned for comparison]",
    xaxis_title="Time (day) — delayed for some countries",
    yaxis_title="COVID-19 total deaths / nb of inhabitants (million)",
    annotations = [dict(xref='paper',
        yref='paper',
        x=0, y=1.1,
        showarrow=False,
        text ='Last update: {} ; Last data: {}'.format(today, df_confirmed['date'].values[-1]))]
)

fig.update_xaxes(nticks = 30)

fig.show()
py.plot(fig, filename = 'deaths-aligned', auto_open=False)
plotly.offline.plot(fig, filename = 'deaths_aligned.html', auto_open=False)
#py.iplot(fig, filename='covid_aligned.html')

KeyboardInterrupt: 

# Dashboard

In [66]:
import chart_studio.dashboard_objs as dashboard
import IPython.display
from IPython.display import Image

my_dboard = dashboard.Dashboard()

box_cases = {
    'type': 'box',
    'boxType': 'plot',
    'fileId': 'worldice:12',
    'title': 'scatter-for-dashboard'
}
box_cases_aligned = {
    'type': 'box',
    'boxType': 'plot',
    'fileId': 'worldice:7',
    'title': 'scatter-for-dashboard'
}
box_deaths = {
    'type': 'box',
    'boxType': 'plot',
    'fileId': 'worldice:5',
    'title': 'scatter-for-dashboard'
}
box_deaths_aligned = {
    'type': 'box',
    'boxType': 'plot',
    'fileId': 'worldice:3',
    'title': 'scatter-for-dashboard',
}
text_for_box="ha"
box_text = {
    'type': 'box',
    'boxType': 'text',
    'text': text_for_box,
    'title': 'Markdown Options for Text Box'
}

my_dboard.insert(box_cases, 1)

my_dboard.insert(box_deaths, 'below', 1)
my_dboard.insert(box_cases_aligned, 'below', 1)

my_dboard.insert(box_deaths_aligned, 'below', 3)

my_dboard['layout']['size'] = 2500
my_dboard['settings']['title'] = 'COVID-19 Stats - @guillaumerozier - data: worldometer'
my_dboard.get_preview()

In [68]:
import chart_studio.plotly as py
py.dashboard_ops.upload(my_dboard, 'COVID-19 Europe Dashboard', auto_open=False)

'https://plot.ly/~worldice/14/covid-19-stats-guillaumerozier-data-worldometer/'

'https://plot.ly/~worldice/24/covid-19-stats-grz/'

In [ ]:
    '''
fig.add_trace(go.Scatter(x=df_confirmed['date'][-last_d:], y=df_confirmed['France'][-last_d:]/pop_fr,
                    mode='lines+markers',
                    name='France'.format("")))

fig.add_trace(go.Scatter(x=df_confirmed['date'][-last_d:], y=df_confirmed['Italy'][-last_d:]/pop_it,
                    mode='lines+markers',
                    name='Italy'))

fig.add_trace(go.Scatter(x=df_confirmed['date'][-last_d:], y=df_confirmed['United Kingdom'][-last_d:]/pop_uk,
                    mode='lines+markers',
                    name='United Kingdom'.format("")))

fig.add_trace(go.Scatter(x=df_confirmed['date'][-last_d:], y=df_confirmed['China'][-last_d:]/pop_ch,
                    mode='lines+markers',
                    name='Chine'.format("")))

fig.add_trace(go.Scatter(x=df_confirmed['date'][-last_d:], y=df_confirmed['Spain'][-last_d:]/pop_sp,
                    mode='lines+markers',
                    name='Spain'.format("")))

fig.add_trace(go.Scatter(x=df_confirmed['date'][-last_d:], y=df_confirmed['Germany'][-last_d:]/pop_ge,
                    mode='lines+markers',
                    name='Germany'.format("")))

fig.add_trace(go.Scatter(x=df_confirmed['date'][-last_d:], y=df_confirmed['Belgium'][-last_d:]/pop_be,
                    mode='lines+markers',
                    name='Belgium'.format("")))
'''